In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


data_dir = '../data/vegetable_images' 


transform = transforms.Compose([
    transforms.Resize(299),  
    transforms.CenterCrop(299), 
    transforms.ToTensor(),  
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


train_dataset = ImageFolder(root=data_dir + '/train', transform=transform)
val_dataset = ImageFolder(root=data_dir + '/validation', transform=transform)

val_dataset_with_noise = ImageFolder(root=data_dir + '/gaussian_noise/validation/100', transform=transform)
test_dataset = ImageFolder(root=data_dir + '/gaussian_noise/test/100', transform=transform)
test_dataset_without_noise = ImageFolder(root=data_dir + '/test', transform=transform)


dataloaders = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True), 
               'val': DataLoader(val_dataset, batch_size=64)
               }

dataloaders_with_noise = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True), 
               'val': DataLoader(val_dataset_with_noise, batch_size=64)
               }


test_loader = DataLoader(test_dataset, batch_size=64)
test_loader_without_noise = DataLoader(test_dataset_without_noise, batch_size=64)




In [4]:
print(len(dataloaders['train'].dataset))
print(len(dataloaders['val'].dataset))

dataset_sizes = {'train' : len(dataloaders['train'].dataset), 'val': len(dataloaders['val'].dataset), 'test': len(test_loader.dataset)}
print(len(test_loader.dataset))
classes  = train_dataset.classes
classes

15000
3000
3000


['Bean',
 'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']

In [5]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /home/sign-lang/.cache/torch/hub/v0.10.0.zip
/home/sign-lang/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /home/sign-lang/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [9]:
num_features = model.fc.in_features


model.fc = torch.nn.Linear(num_features, len(classes))

In [10]:
model = model.to(device)

In [16]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader_without_noise:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.logits, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 6.9 %
Accuracy of Bean: 17.0 %
Accuracy of Bitter_Gourd: 3.0 %
Accuracy of Bottle_Gourd: 22.5 %
Accuracy of Brinjal: 0.5 %
Accuracy of Broccoli: 2.0 %
Accuracy of Cabbage: 4.0 %
Accuracy of Capsicum: 11.0 %
Accuracy of Carrot: 11.0 %
Accuracy of Cauliflower: 6.5 %
Accuracy of Cucumber: 1.5 %
